In [3]:
import json

# 002 - 필지, 010 - 건물

with open("AL_11_D010_20220702.json", encoding="utf-8") as f:
    map_data_all = json.load(f)

def flatten_list(list_instance):
    if list_instance == []:
        return list_instance
    if isinstance(list_instance[0], list):
        return flatten_list(list_instance[0]) + flatten_list(list_instance[1:])
    return list_instance[:1] + flatten_list(list_instance[1:])

In [4]:
map_data_filtered = {"type": map_data_all["type"], "features": []}

for each_building in map_data_all["features"]:

    coordinates = each_building["geometry"]["coordinates"]
    flattened_coordinates = flatten_list(coordinates)
    lon_s, lat_s = flattened_coordinates[0], flattened_coordinates[1]

    # if (not 125 < lon_s < 130) or (not 34 < lat_s < 39):
    #     # 기준 잡을 때 위도 경도 맞는지 체크 - 0개
    #     print("위도 경도 맞지 않음")
    #     continue
    if each_building["geometry"]["type"] != "Polygon":
        # MultiPolygon 4개 존재.
        print("폴리곤 아님")
        continue

    map_data_filtered["features"].append(each_building)

폴리곤 아님
폴리곤 아님
폴리곤 아님
폴리곤 아님


In [11]:
y_min, x_min, y_max, x_max = 100000000000000, 100000000000000, -100000000000000, -100000000000000

for each_building in map_data_filtered["features"]:

    coordinates = each_building["geometry"]["coordinates"]
    flattened_coordinates = flatten_list(coordinates)
    x_s, y_s = flattened_coordinates[0], flattened_coordinates[1]

    # 이상한 데이터 섞여있음
    if x_s > y_s:
        continue

    if y_min > y_s:
        y_min = y_s
    
    if x_min > x_s:
        x_min = x_s
    
    if y_max < y_s:
        y_max = y_s

    if x_max < x_s:
        x_max = x_s

# 10 * 10으로 나누기 위해 최대 최소 확인
x_min, y_min, x_max, y_max

(179404.39400000032, 436476.5030000005, 219589.24000000022, 466307.8200000003)

In [16]:
division_num = 100

x_interval = (x_max - x_min) / division_num
y_interval = (y_max - y_min) / division_num

x_list = [x_min + x_interval * i for i in range(division_num + 1)]
y_list = [y_min + y_interval * i for i in range(division_num + 1)]

divided_map_data = {}
for building_index, each_building in enumerate(map_data_filtered["features"]):
    coordinates = each_building["geometry"]["coordinates"]
    flattened_coordinates = flatten_list(coordinates)
    x_s, y_s = flattened_coordinates[0], flattened_coordinates[1]

    for i in range(division_num):
        if y_list[i] <= y_s < y_list[i + 1]:
            for j in range(division_num):
                if x_list[j] <= x_s < x_list[j + 1]:

                    if (y_list[i], x_list[i], y_list[i + 1], x_list[i + 1]) in divided_map_data:
                        divided_map_data[(y_list[i], x_list[i], y_list[i + 1], x_list[i + 1])]["features"].append(each_building)
                    else:
                        divided_map_data[(y_list[i], x_list[i], y_list[i + 1], x_list[i + 1])] = {"type": map_data_all["type"], "features": [each_building]}


In [ ]:
[len(divided_map_data[key]["features"]) for key in divided_map_data]

In [21]:
for loc_key in divided_map_data:
    with open(f"building_data_divided/{loc_key[0]}_{loc_key[1]}_{loc_key[2]}_{loc_key[3]}.json", "w", encoding="utf-8") as f:
        json.dump(divided_map_data[loc_key], f)